In [11]:
import struct
import os
# Open the file in binary mode

safetensor_file = "./net.safetensors"

def get_meta_size(file):
    with open(file, 'rb') as f:
        # Read the first 8 bytes
        bytes_data = f.read(8)
        # print them as uint8 integers, convert hex to decimal
        print(" ".join([str(x) for x in bytes_data]))
        # Unpack the bytes to a 64-bit unsigned integer
        meta_size = struct.unpack('Q', bytes_data)[0]
        return meta_size

print(get_meta_size(safetensor_file))

240 16 3 0 0 0 0 0
200944


In [12]:
from safetensors.numpy import load_file

loaded = load_file(safetensor_file)

In [15]:
# for ech k, v, print the size of v in megabytes
max_weight_size = 0
for k, v in loaded.items():
    print(k, v.shape, v.dtype, v.nbytes / 1024 / 1024, "MB")
    max_weight_size = max(max_weight_size, v.nbytes / 1024 / 1024)
print("Max weight size:", max_weight_size, "MB")

alphas_cumprod (1000,) float32 0.003814697265625 MB
cond_stage_model.transformer.text_model.embeddings.position_embedding.vocab_counter (1, 1, 77) float32 0.000293731689453125 MB
cond_stage_model.transformer.text_model.embeddings.position_embedding.weight (77, 768) float32 0.2255859375 MB
cond_stage_model.transformer.text_model.embeddings.token_embedding.vocab_counter (1, 1, 49408) float32 0.1884765625 MB
cond_stage_model.transformer.text_model.embeddings.token_embedding.weight (49408, 768) float32 144.75 MB
cond_stage_model.transformer.text_model.encoder.layers.0.layer_norm1.bias (768,) float32 0.0029296875 MB
cond_stage_model.transformer.text_model.encoder.layers.0.layer_norm1.weight (768,) float32 0.0029296875 MB
cond_stage_model.transformer.text_model.encoder.layers.0.layer_norm2.bias (768,) float32 0.0029296875 MB
cond_stage_model.transformer.text_model.encoder.layers.0.layer_norm2.weight (768,) float32 0.0029296875 MB
cond_stage_model.transformer.text_model.encoder.layers.0.mlp.f